# Comparing actuals with benchmark, using pandas with SQL

We can achieve the same results we had in the ActualsVsBenchmark notebook, leveraging on the SQL skills we already acquired. In pandas, we can use the sqlite engine to do some dataframe processing. The sqlite engine is not as standards compliant as we might wish, but nevertheless it can come to goo use. Let's bring it on.

In [1]:
from pandasql import *
import pandas as pd

First, we will import the data from our external sources.

In [2]:
import html5lib

df_bm = pd.read_html('dummytxt.html')[0]
df_bm.head(3)

,Plaats,Maand,18-24,25-39,40-58,59-69,70-100
0,Best,jan,68,76,76,122,108
1,Eindhoven,jan,93,60,60,72,137
2,Helmond,jan,70,99,68,70,124


In [3]:
df_ord = pd.read_excel('opdracht.xlsx', sheetname='sales')
df_ord.head(3)

,nr,datum,bedrag
0,1009,2017-01-03,106
1,1012,2017-01-03,55
2,1006,2017-01-09,37


In [4]:
df_cust = pd.read_csv('Klanten.csv', skiprows=8, parse_dates=[3])
df_cust.head(3)

,nr,naam,gesl,gebdat,wpl,vooropl,tel
0,1004,tinus,m,1983-01-26,nue,VO,06-52477478
1,1005,henk,m,1961-04-12,nue,HBO,06-49945368
2,1006,lars,m,1967-06-07,ehv,VO,06-42766537


First we have to unpivot the age segement values. In plain pandas we used a melt() for this. In SQL we can do this using unions of partial tables. We have as many partial tables as we have columns to unpivot. Less fancy but equally effective.

Note that sqlite does not provide the standard SQL unpivot facility.

In [5]:
qry = """

select Plaats, Maand,
    '18-24' as Segment,
    [18-24] as Value
from df_bm
union all
select Plaats, Maand,
    '25-39' as Segment,
    [25-39] as Value
from df_bm
union all
select Plaats, Maand,
    '40-58' as Segment,
    [40-58] as Value
from df_bm
union all
select Plaats, Maand,
    '59-69' as Segment,
    [59-69] as Value
from df_bm
union all
select Plaats, Maand,
    '70-100' as Segment,
    [70-100] as Value
from df_bm

"""

df_bm_ext = sqldf(qry)
df_bm_ext.head(3)

,Plaats,Maand,Segment,Value
0,Best,jan,18-24,68
1,Eindhoven,jan,18-24,93
2,Helmond,jan,18-24,70


Map values in customers dataframe to values in benchmark dataframe in order to make them joinable.

In [6]:
qry = """

select nr,
    case wpl
        when 'ehv' then 'Eindhoven'
        when 'hmd' then 'Helmond'
        when 'nue' then 'Nuenen'
        when 'vhv' then 'Veldhoven'
    end as woonplaats,
    cast(strftime('%Y.%m%d', 'now') - strftime('%Y.%m%d', gebdat) as int) as leeftijd
from df_cust

"""

df_cust_ext = sqldf(qry)
df_cust_ext.head(3)

,nr,woonplaats,leeftijd
0,1004,Nuenen,34
1,1005,Nuenen,56
2,1006,Eindhoven,50


Merge order data with customer data using a common SQL inner join and group it on a monthly base in one swift move.

In [7]:
qry = """

select woonplaats,
    case strftime('%m', datum)
        when '01' then 'jan'
        when '02' then 'feb'
        when '03' then 'mrt'
    end as maand,
    case
        when leeftijd < 25 then '18-24'
        when leeftijd < 40 then '25-39'
        when leeftijd < 59 then '40-58'
        when leeftijd < 70 then '59-69'
        else '70-100'
    end as segment,
    sum(bedrag) as omzet
from df_cust_ext c
join df_ord o
    on o.nr = c.nr
group by 1, 2, 3

"""

df_ord_ext = sqldf(qry)
df_ord_ext.head(3)

,woonplaats,maand,segment,omzet
0,Eindhoven,feb,59-69,255
1,Eindhoven,jan,40-58,313
2,Eindhoven,jan,59-69,110


Now let's stitch everything together using an outer join:

In [8]:
qry = """

select b.*, omzet as 'Omzet'
from df_bm_ext b
left outer join df_ord_ext o
    on Plaats = woonplaats
    and b.Maand = o.maand
    and b.Segment = o.segment

"""

df = sqldf(qry)
df[df.Omzet.notnull()]

,Plaats,Maand,Segment,Value,Omzet
27,Nuenen,jan,25-39,68,200.0
28,Veldhoven,jan,25-39,105,285.0
34,Veldhoven,feb,25-39,123,199.0
37,Eindhoven,mrt,25-39,92,12.0
40,Veldhoven,mrt,25-39,96,369.0
49,Eindhoven,jan,40-58,60,313.0
50,Helmond,jan,40-58,68,260.0
51,Nuenen,jan,40-58,85,235.0
52,Veldhoven,jan,40-58,68,106.0
56,Helmond,feb,40-58,109,279.0


You can now go on with your visualization just as in the notebook that used plain pandas to do all the merging.

Just for the heck of it, let's capture all different parts in one single SQL statement:

In [9]:
qry = """

with bm as (
select Plaats, Maand,
    '18-24' as Segment,
    [18-24] as Value
from df_bm
union all
select Plaats, Maand,
    '25-39' as Segment,
    [25-39] as Value
from df_bm
union all
select Plaats, Maand,
    '40-58' as Segment,
    [40-58] as Value
from df_bm
union all
select Plaats, Maand,
    '59-69' as Segment,
    [59-69] as Value
from df_bm
union all
select Plaats, Maand,
    '70-100' as Segment,
    [70-100] as Value
from df_bm
),
cust as (
select nr,
    case wpl
        when 'ehv' then 'Eindhoven'
        when 'hmd' then 'Helmond'
        when 'nue' then 'Nuenen'
        when 'vhv' then 'Veldhoven'
    end as woonplaats,
    cast(strftime('%Y.%m%d', 'now') - strftime('%Y.%m%d', gebdat) as int) as leeftijd
from df_cust
),
ord as (
select woonplaats,
    case strftime('%m', datum)
        when '01' then 'jan'
        when '02' then 'feb'
        when '03' then 'mrt'
    end as maand,
    case
        when leeftijd < 25 then '18-24'
        when leeftijd < 40 then '25-39'
        when leeftijd < 59 then '40-58'
        when leeftijd < 70 then '59-69'
        else '70-100'
    end as segment,
    sum(bedrag) as omzet
from df_cust_ext c
join df_ord o
    on o.nr = c.nr
group by 1, 2, 3
)
select b.*, omzet as 'Omzet'
from df_bm_ext b
left outer join df_ord_ext o
    on Plaats = woonplaats
    and b.Maand = o.maand
    and b.Segment = o.segment

"""
df1 = sqldf(qry)
df1[df1.Omzet.notnull()]

,Plaats,Maand,Segment,Value,Omzet
27,Nuenen,jan,25-39,68,200.0
28,Veldhoven,jan,25-39,105,285.0
34,Veldhoven,feb,25-39,123,199.0
37,Eindhoven,mrt,25-39,92,12.0
40,Veldhoven,mrt,25-39,96,369.0
49,Eindhoven,jan,40-58,60,313.0
50,Helmond,jan,40-58,68,260.0
51,Nuenen,jan,40-58,85,235.0
52,Veldhoven,jan,40-58,68,106.0
56,Helmond,feb,40-58,109,279.0
